# Convergence guarantees - ENCO

This notebook contains an example of how we can calculate the convergence conditions for ENCO on a simple graph. This notebook assumes the following graph structure: $X_1\to X_2\to X_3$. The conditional distributions are specified below, and can be alternated freely. The code is fixed for the specific graph structure, but it can be extended to other graph structures by changing the calculation of the corresponding conditional distributions.

In [1]:
import numpy as np

## Conditional distributions

In [2]:
p_1 = np.array([0.3, 0.7])                  # p(X_1) - First value: p(X_1=0), Second value: p(X_1=1)
p_2_1 = np.array([[0.6, 0.4], [0.4, 0.6]])  # p(X_2|X_1) - First axis: X_1, Second axis: X_2
p_3_2 = np.array([[0.2, 0.8], [0.8, 0.2]])  # p(X_3|X_2) - First axis: X_2, Second axis: X_3

In [3]:
p_interv = np.array([0.5, 0.5])  # Distribution to use for interventions instead of original conditionals

### Determining relevant conditionals

In [4]:
p_joint = p_1[:,None,None] * p_2_1[:,:,None] * p_3_2[None,:,:]  # p(X_1,X_2,X_3) - First axis: X_1, Second axis: X_2, Third axis: X_3
print(f'{p_joint=}')
print(f'{p_joint.sum()=}')

p_joint=array([[[0.036, 0.144],
        [0.096, 0.024]],

       [[0.056, 0.224],
        [0.336, 0.084]]])
p_joint.sum()=1.0


In [5]:
p_2 = (p_1[:,None] * p_2_1).sum(axis=0)  # p(X_2)
p_3 = (p_2[:,None] * p_3_2).sum(axis=0)  # p(X_3)
print(f'{p_2=}')
print(f'{p_3=}')

p_2=array([0.46, 0.54])
p_3=array([0.524, 0.476])


In [6]:
p_2_3 = (p_3_2 * p_2[:,None] / p_3[None,:]).transpose()  # p(X_2|X_3) - First axis: X_3, Second axis: X_2
print(f'{p_2_3=}')

p_2_3=array([[0.17557252, 0.82442748],
       [0.77310924, 0.22689076]])


In [7]:
p_3_1 = (p_2_1[:,:,None] * p_3_2[None,:,:]).sum(axis=1)  # p(X_3|X_1) - First axis: X_1, Second axis: X_3
print(f'{p_3_1=}')

p_3_1=array([[0.44, 0.56],
       [0.56, 0.44]])


In [8]:
p_2_1_3 = np.transpose(p_3_2[None,:,:] * p_2_1[:,:,None] / p_3_1[:,None,:], (0, 2, 1))  # p(X_2|X_1,X_3) - First axis: X_1, Second axis: X_3, Third axis: X_2
print(f'{p_2_1_3=}')

p_2_1_3=array([[[0.27272727, 0.72727273],
        [0.85714286, 0.14285714]],

       [[0.14285714, 0.85714286],
        [0.72727273, 0.27272727]]])


In [9]:
def expect_diff(joint_prob, prob1, prob2):  # Expected log-likelihood difference
    return (joint_prob * (np.log(prob1) - np.log(prob2))).sum()

### Edge $X_1\to X_2$

_Condition 1 - Part 1_

In [10]:
p_interv_1_2 = p_interv[:,None] * p_2_1  # p(do(X_1),X_2) - First axis X_1, Second axis X_2
print(f'{p_interv_1_2=}')

p_interv_1_2=array([[0.3, 0.2],
       [0.2, 0.3]])


In [11]:
expect_diff(p_interv_1_2, p_2_1, p_2[None,:])

0.023345797452150232

_Condition 1 - Part 2_

In [12]:
p_interv_1_2_3 = p_interv[:,None,None] * p_2_1[:,:,None] * p_3_2[None,:,:]  # p(do(X_1),X_2,X_3)
print(f'{p_interv_1_2_3=}')

p_interv_1_2_3=array([[[0.06, 0.24],
        [0.16, 0.04]],

       [[0.04, 0.16],
        [0.24, 0.06]]])


In [13]:
expect_diff(p_interv_1_2_3.transpose((0, 2, 1)), p_2_1_3, p_2_3[None,:,:])

0.014975087998617024

_Condition 3_

In [14]:
p_interv_3_1_2 = p_interv[:,None,None] * p_1[None,:,None] * p_2_1[None,:,:]  # p(do(X_3),X_1,X_2)
print(f'{p_interv_3_1_2=}')

p_interv_3_1_2=array([[[0.09, 0.06],
        [0.14, 0.21]],

       [[0.09, 0.06],
        [0.14, 0.21]]])


In [15]:
expect_diff(p_interv_1_2_3, p_2_1[:,:,None], p_2[None,:,None])

0.02334579745215022

In [16]:
expect_diff(p_interv_3_1_2, p_2_1[None,:,:], p_2[None,None,:])

0.016932091449143066

In [17]:
0.5 * expect_diff(p_interv_1_2_3, p_2_1[:,:,None], p_2[None,:,None]) + 0.5 * expect_diff(p_interv_3_1_2, p_2_1[None,:,:], p_2[None,None,:])

0.020138944450646644

### Edge $X_2\to X_3$

_Condition 1 - Part 1_

In [18]:
p_interv_2_3 = p_interv[:,None] * p_3_2  # p(do(X_2),X_3)
print(f'{p_interv_2_3=}')

p_interv_2_3=array([[0.1, 0.4],
       [0.4, 0.1]])


In [19]:
expect_diff(p_interv_2_3, p_3_2, p_3[None,:])

0.19389808616771806

_Condition 1 - Part 2_

In [20]:
p_interv_2_3_1 = p_interv[:,None,None] * p_3_2[:,:,None] * p_1[None,None,:]  # p(do(X_2),X_3,X_1)
print(f'{p_interv_2_3_1=}')

p_interv_2_3_1=array([[[0.03, 0.07],
        [0.12, 0.28]],

       [[0.12, 0.28],
        [0.03, 0.07]]])


In [21]:
expect_diff(p_interv_2_3_1.transpose((0,2,1)), p_3_2[:,None,:], p_3_1[None,:,:])

0.19999710012319816

_Condition 3 - Part 1_

In [22]:
int1 = expect_diff(p_interv_1_2_3, p_3_2[None,:,:], p_3[None,None,:])
int1

0.19389808616771811

In [23]:
int2 = expect_diff(p_interv_2_3_1, p_3_2[:,:,None], p_3[None,:,None])
int2

0.1938980861677181

In [24]:
0.5 * int1 + 0.5 * int2

0.1938980861677181

_Condition 3 - Part 2_

In [25]:
int1 = expect_diff(p_interv_1_2_3, p_3_2[None,:,:], p_3_1[:,None,:])
int1

0.1855273767141849

In [26]:
int2 = expect_diff(p_interv_2_3_1.transpose((0,2,1)), p_3_2[:,None,:], p_3_1[None,:,:])
int2

0.19999710012319816

In [27]:
0.5 * int1 + 0.5 * int2

0.19276223841869153

## Ancestor-Descendant pair $X_1,X_3$

_Condition 1 - Part 1_

In [28]:
p_interv_1_3 = p_interv_1_2_3.sum(axis=1)  # p(do(X_1),X_3)
print(f'{p_interv_1_3=}')

p_interv_1_3=array([[0.22, 0.28],
       [0.28, 0.22]])


In [29]:
expect_diff(p_interv_1_3, p_3_1, p_3[None,:])

0.008370709453533205

_Condition 1 - Part 2_

In [30]:
expect_diff(p_interv_1_2_3, p_3_2[None,:,:], p_3_2[None,:,:])

0.0